## SOM to reduce numSpikes, then t-SNE to embed and HDBSCAN to cluster

In [1]:
import pandas as pd
import time as time
import numpy as np
import os
from matplotlib import pyplot as plt
from jaratoolbox import loadopenephys
import sys
import SOMPY

/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
%matplotlib inline

In [3]:
# Use a giant dataset, 1mil spikes

animalName='adap020'
ephysLoc = '/home/nick/data/ephys/'
ephysPath = os.path.join(ephysLoc, animalName)
ephysFn='2016-05-25_16-33-09'
tetrode=2
spikesFn = os.path.join(ephysPath, ephysFn, 'Tetrode{}.spikes'.format(tetrode))
dataSpikes = loadopenephys.DataSpikes(spikesFn)

In [4]:
len(dataSpikes.samples)

1117928

In [5]:
(numSpikes, numChans, numSamples) = np.shape(dataSpikes.samples)

allWaves = dataSpikes.samples.reshape(numSpikes, numChans*numSamples)

In [ ]:
reload(SOMPY)

In [6]:
import timeit
start_time = timeit.default_timer()

msz0 = 30
msz1 = 30

sm = SOMPY.sompy.SOM(allWaves, neighborhood=SOMPY.neighborhood.GaussianNeighborhood(), normalizer=SOMPY.normalization.VarianceNormalizator(), mapsize = [msz0, msz1], initialization='pca', name='sm')
sm.train(n_job = 1, shared_memory = 'yes')

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

 Training...
 pca_linear_initialization took: 8.410000 seconds
 Rough training...
 radius_ini: 4.000000 , radius_final: 1.000000, trainlen: 1

 epoch: 1 ---> elapsed time:  44.610000, quantization error: 10.623795

 Finetune training...
 radius_ini: 1.000000 , radius_final: 1.000000, trainlen: 1

 epoch: 1 ---> elapsed time:  44.841000, quantization error: 10.427436

 Final quantization error: 10.427436
 train took: 113.828000 seconds


ELAPSED TIME: 1.93403821786 mins


In [7]:
codebookVecs = sm.codebook.matrix


In [ ]:
from sklearn.manifold import TSNE

In [ ]:
start_time = timeit.default_timer()

model = TSNE(n_components=2, method='barnes_hut', verbose=20, n_iter=1000)
Y = model.fit_transform(codebookVecs)

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

In [ ]:
import hdbscan

In [ ]:
start_time = timeit.default_timer()

clus = hdbscan.HDBSCAN(min_cluster_size=10)
cluster_labels = clus.fit_predict(Y)

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

In [ ]:
plt.figure(figsize=(10, 10))
uniqueLabels = np.unique(cluster_labels)
colors = plt.cm.Paired(np.linspace(0, 1, len(uniqueLabels)))
for indLabel, label in enumerate(np.unique(cluster_labels)):
    plt.hold(1)
    indsThisLabel = np.flatnonzero(cluster_labels==label)
    plt.plot(Y[indsThisLabel, 0], Y[indsThisLabel, 1], '.', color=colors[indLabel])

Bad clustering with HDBSCAN

In [ ]:
cluster_labels = sm.cluster()

In [ ]:
plt.figure(figsize=(10, 10))
uniqueLabels = np.unique(cluster_labels)
colors = plt.cm.Paired(np.linspace(0, 1, len(uniqueLabels)))
for indLabel, label in enumerate(np.unique(cluster_labels)):
    plt.hold(1)
    indsThisLabel = np.flatnonzero(cluster_labels==label)
    plt.plot(Y[indsThisLabel, 0], Y[indsThisLabel, 1], '.', color=colors[indLabel])

In [ ]:
start_time = timeit.default_timer()

dataClusters = cluster_labels[sm.project_data(allWaves)]

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

In [ ]:
plt.figure(figsize=(10, 20))
labels = dataClusters
rav = allWaves.ravel()
maxv = np.percentile(rav, 99.95)
minv = np.percentile(rav, 0.05)
for indLabel, label in enumerate(uniqueLabels):
    plt.subplot(len(uniqueLabels), 1, indLabel+1)
    clusterWaves = allWaves[labels==label]

    spikesToPlot = np.random.randint(len(clusterWaves),size=50)

    for wave in clusterWaves[spikesToPlot]:
        plt.plot(wave, '-', alpha=0.5, color=colors[indLabel])
        plt.hold(1)
    plt.plot(clusterWaves.mean(0), 'k', lw=2, zorder=10)
    plt.ylim([minv, maxv])
    plt.axvline(x=40, lw=3, color='k')
    plt.axvline(x=80, lw=3, color='k')
    plt.axvline(x=120, lw=3, color='k')

In [ ]:
from jaratoolbox import spikesorting

In [ ]:
dataSpikes.clusters = dataClusters

GAIN = 5000.0
SAMPLING_RATE=30000.0
dataSpikes.samples = ((dataSpikes.samples - 32768.0) / GAIN) * 1000.0
dataSpikes.timestamps = dataSpikes.timestamps/SAMPLING_RATE

cr = spikesorting.ClusterReportFromData(dataSpikes,
                                        outputDir='/home/nick/Desktop',
                                        filename='som_largedata{}{}{}.png'.format(animalName, ephysFn,'Tetrode{}'.format(tetrode) ))


Still not too good. We need to figure out a better way to cluster the codebook vectors. 

In [ ]:
import sklearn

In [ ]:
cluster_labels = sklearn.cluster.KMeans(n_clusters=10).fit_predict(Y)

In [ ]:
plt.figure(figsize=(10, 10))
uniqueLabels = np.unique(cluster_labels)
colors = plt.cm.Paired(np.linspace(0, 1, len(uniqueLabels)))
for indLabel, label in enumerate(np.unique(cluster_labels)):
    plt.hold(1)
    indsThisLabel = np.flatnonzero(cluster_labels==label)
    plt.plot(Y[indsThisLabel, 0], Y[indsThisLabel, 1], '.', color=colors[indLabel])

In [ ]:
start_time = timeit.default_timer()

dataClusters = cluster_labels[sm.project_data(allWaves)]

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

In [ ]:
start_time = timeit.default_timer()

dataClusters = cluster_labels[sm.project_data_balltree(allWaves)]

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

In [ ]:
plt.figure(figsize=(10, 20))
labels = dataClusters
rav = allWaves.ravel()
maxv = np.percentile(rav, 99.95)
minv = np.percentile(rav, 0.05)
for indLabel, label in enumerate(uniqueLabels):
    plt.subplot(len(uniqueLabels), 1, indLabel+1)
    clusterWaves = allWaves[labels==label]

    spikesToPlot = np.random.randint(len(clusterWaves),size=50)

    for wave in clusterWaves[spikesToPlot]:
        plt.plot(wave, '-', alpha=0.5, color=colors[indLabel])
        plt.hold(1)
    plt.plot(clusterWaves.mean(0), 'k', lw=2, zorder=10)
    plt.ylim([minv, maxv])
    plt.axvline(x=40, lw=3, color='k')
    plt.axvline(x=80, lw=3, color='k')
    plt.axvline(x=120, lw=3, color='k')

In [ ]:
dataSpikes.samples #Already in uV

In [ ]:
dataSpikes.clusters = dataClusters

cr = spikesorting.ClusterReportFromData(dataSpikes,
                                        outputDir='/home/nick/Desktop',
                                        filename='som_largedata_TSNE_KMEANS-{}-{}-{}.png'.format(animalName, ephysFn,'Tetrode{}'.format(tetrode) ))


## New approach to clustering the codebook vectors

This paper:

http://publications.mi.fu-berlin.de/133/1/SC-99-38.pdf

talks about an approach to clustering the codebook vectors that is automatic and also takes into account the topology of the SOM, which is one of the reasons to use a SOM in the first place. 

Another one:
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.387.7910&rep=rep1&type=pdf

In [ ]:
import SOMPY.hitmap
import SOMPY.umatrix

In [ ]:
np.shape(codebookVecs)

In [9]:
import sklearn.cluster

In [10]:
agg = sklearn.cluster.AgglomerativeClustering(n_clusters=12)
cluster_labels = agg.fit_predict(codebookVecs)

In [ ]:
plt.figure(figsize=(10, 10))
uniqueLabels = np.unique(cluster_labels)
colors = plt.cm.Paired(np.linspace(0, 1, len(uniqueLabels)))
for indLabel, label in enumerate(np.unique(cluster_labels)):
    plt.hold(1)
    indsThisLabel = np.flatnonzero(cluster_labels==label)
    plt.plot(Y[indsThisLabel, 0], Y[indsThisLabel, 1], '.', color=colors[indLabel])

In [13]:
start_time = timeit.default_timer()

dataClustersSlow = cluster_labels[sm.project_data(allWaves)]

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

ELAPSED TIME: 3.30719913244 mins


In [12]:
start_time = timeit.default_timer()

dataClusters = cluster_labels[sm.project_data_balltree(allWaves)]

elapsed = timeit.default_timer() - start_time
print 'ELAPSED TIME: {} mins'.format(elapsed/60)

ELAPSED TIME: 1.89144690037 mins


In [15]:
dataClustersSlow

array([1, 8, 5, ..., 9, 1, 9])

In [23]:
sum(dataClusters==3)/np.double(len(dataClusters)) # I guess the balltree approx is bad. 

0.99999910548800997

In [ ]:
x = 1

In [24]:
%qtconsole

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
